In [ ]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed as Receiver
from deep_orderbook.shaper import BookShaper

import asyncio
import pandas as pd
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["BTC-USD", "ETH-USD", "ETH-BTC", ]

In [ ]:
async with Receiver(markets=MARKETS) as receiver:
    await asyncio.sleep(5)
#receiver = await Writer.create(markets=MARKETS, data_folder='../crypto-trading/data')
#wrting = asyncio.create_task(receiver.run_writer(save_period_minutes=10))

In [ ]:
b = receiver.depth_managers['BTC-USD'].get_bids()
pd.DataFrame(b, columns=['price', 'size']).astype(np.float64).set_index('price')#.cumsum().plot()

In [ ]:
a = receiver.depth_managers['BTC-USD'].get_asks()
pd.DataFrame(a, columns=['price', 'size']).astype(np.float64).set_index('price')

In [ ]:
receiver.trade_tapes['BTC-USD']

In [ ]:
multi_replay = receiver.multi_generator(MARKETS)
_ = await multi_replay.__anext__()
_

In [ ]:
import aioitertools
genarr = BookShaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await aioitertools.next(genarr)
_

In [ ]:
genacc = BookShaper.accumulate_array(genarr, markets=MARKETS)
_ = await aioitertools.next(genacc)
_

In [ ]:
async with Receiver(markets=MARKETS) as receiver:
    multi_replay = receiver.multi_generator(MARKETS)
    genarr = BookShaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
    genacc = BookShaper.accumulate_array(genarr, markets=MARKETS)
    async for toshow in BookShaper.images(accumulated_arrays=genacc, every=1, LENGTH=512):
        plt.figure(figsize=(15,11))
        plt.imshow(toshow, origin="upper")
        clear_output(wait=True)
        plt.show()

In [ ]:
#task = asyncio.create_task(images(accumulated_arrays=genacc, every=1, LENGTH=512))